In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datacompy
import os
import numpy as np
from joblib import dump, load
import pickle
from scipy.io import arff

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import LabelEncoder

# modele + Smote
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import NearMiss, RandomUnderSampler, EditedNearestNeighbours, TomekLinks, NeighbourhoodCleaningRule, InstanceHardnessThreshold, OneSidedSelection, CondensedNearestNeighbour
from ctgan import CTGAN

### Data Preprocessing

Focuse on service == http, drop columns which have the same values in all rows ('protocol_type', 'land', 'wrong_fragment', 'urgent', 'num_failed_logins', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_outbound_cmds', 'is_host_login', 'is_guest_login'). drop columns "index" and "flag" after dummies this column. Swap class from values "normaly" & "anomaly" to 1 & 0. 

In [78]:
#read data & use only http services
df = pd.read_csv("..\\data\\IPS\\Train_data.csv")
df = df[df['service'].str.contains('http')]
#drop columns service and protocol_type and index
df = df.drop(columns=["service"])
df = df.reset_index()
df = df.drop(columns=["protocol_type"])
df = df.drop(columns=["index"])

In [80]:
#split column flag
df_dumm = pd.get_dummies(df["flag"], prefix="flag", dtype=int)
col = df.columns.get_loc('flag')+1
for col_name in reversed(df_dumm.columns):  
    df.insert(col, col_name, df_dumm[col_name])
df = df.drop(columns=["flag"])

#drop columns with one value f.g. 1 in all rows
df['class'] = df['class'].replace({'normal': 1, 'anomaly': 0})
df = df.drop(columns=['land', 'wrong_fragment', 'urgent', 'num_failed_logins', 'su_attempted',
       'num_root', 'num_file_creations', 'num_shells', 'num_outbound_cmds',
       'is_host_login', 'is_guest_login'])
df.to_excel("D:\\ml\\xgboost-main\\data\\IPS\\http_data.xlsx")

#print column class distribution
colum = df['class']
print(colum.shape)
pd.Series(colum).value_counts().plot.bar()

In [6]:
df_test = pd.read_csv("..\\data\\IPS\\Test_data.csv")
df_test = df_test[df_test['service'].str.contains('http')]
colum = df_test['service'].value_counts()
print(colum)

service
http        7853
http_443      36
Name: count, dtype: int64
